In [1]:
import requests

url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

querystring = {"sort":"relevance","prop_type":"single_family","city":"Los Angeles","limit":"5000","offset":"0","state_code":"CA"}

headers = {
    'x-rapidapi-host': "realtor.p.rapidapi.com",
    'x-rapidapi-key': "ccd80db736mshcfc9234f86f9d33p12faf8jsn0eb2e033ed0c"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response_json=response.json()

print(response_json.keys())

dict_keys(['meta', 'properties'])


In [0]:
properties = response_json['properties']

In [3]:
properties[1].keys()

dict_keys(['property_id', 'listing_id', 'rdc_web_url', 'prop_type', 'virtual_tour', 'address', 'branding', 'prop_status', 'price', 'baths_full', 'baths', 'beds', 'building_size', 'agents', 'office', 'last_update', 'client_display_flags', 'lead_forms', 'photo_count', 'thumbnail', 'page_no', 'rank', 'list_tracking', 'lot_size', 'mls'])

In [4]:
import pandas as pd
CITY_NAME_DF = pd.DataFrame(properties)
CITY_NAME_DF.head()

,property_id,listing_id,rdc_web_url,prop_type,virtual_tour,address,branding,prop_status,price,baths_full,baths,beds,building_size,agents,office,last_update,client_display_flags,lead_forms,photo_count,thumbnail,page_no,rank,list_tracking,lot_size,mls,baths_half
0,M1947154472,2916016445,https://www.realtor.com/realestateandhomes-det...,single_family,{'href': 'https://aiHomeTours.vr-360-tour.com/...,"{'city': 'Los Angeles', 'line': '6207 Keniston...",{'listing_office': {'list_item': {'name': 'Vis...,for_sale,850000,2.0,2.0,3,"{'size': 1425, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '2798574',...","{'id': '814a606bd6cf7b95041dac65a962ea3e', 'na...",2020-05-07T17:29:48Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",35,https://ap.rdcpix.com/31823bb68d78c4816107968c...,1,1,type|property|data|prop_id|1947154472|list_id|...,"{'size': 5649, 'units': 'sqft'}","{'name': 'CRMLS', 'id': 'SB20086930', 'plan_id...",NaN
1,M1877462085,2916014821,https://www.realtor.com/realestateandhomes-det...,single_family,{'href': 'https://my.matterport.com/show/?m=Qa...,"{'city': 'Los Angeles', 'line': '5607 S Wilton...",{'listing_office': {'list_item': {'name': 'Cen...,for_sale,624900,2.0,2.0,3,"{'size': 1268, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3336787',...","{'id': 'e462002e29be4a2906dcd373970a17d6', 'na...",2020-05-07T16:41:18Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",38,https://ap.rdcpix.com/f547d2f4ec6199ad18404a81...,1,2,type|property|data|prop_id|1877462085|list_id|...,"{'size': 5470, 'units': 'sqft'}","{'name': 'CRMLS', 'id': 'DW20087349', 'plan_id...",NaN
2,M2313200155,2916013990,https://www.realtor.com/realestateandhomes-det...,single_family,NaN,"{'city': 'Sun Valley', 'line': '9169 Oneida Av...",{'listing_office': {'list_item': {'name': 'Con...,for_sale,610000,2.0,2.0,3,"{'size': 1300, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '2898583',...","{'id': 'b283878788c48bd0582c2904f7b37091', 'na...",2020-05-07T18:10:19Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",19,https://ap.rdcpix.com/2684aa45f3c149e79454e688...,1,3,type|property|data|prop_id|2313200155|list_id|...,"{'size': 10487, 'units': 'sqft'}","{'name': 'VenturaCounty', 'id': '220004592', '...",NaN
3,M2358860381,2916012921,https://www.realtor.com/realestateandhomes-det...,single_family,NaN,"{'city': 'West Los Angeles', 'line': '11230 Pi...",{'listing_office': {'list_item': {'name': 'So ...,for_sale,1999000,4.0,4.0,3,"{'size': 2573, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3079822',...","{'id': '89ed10a927d46ead43daed4fb7c8974a', 'na...",2020-05-07T15:30:29Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",20,https://ap.rdcpix.com/3af23b1383d0db73b94ea040...,1,4,type|property|data|prop_id|2358860381|list_id|...,"{'size': 6026, 'units': 'sqft'}","{'name': 'CRMLS', 'id': 'RS20087327', 'plan_id...",NaN
4,M2263164590,2916012408,https://www.realtor.com/realestateandhomes-det...,single_family,NaN,"{'city': 'Los Angeles', 'line': '5011 S Budlon...",{'listing_office': {'list_item': {'name': 'Rea...,for_sale,739000,2.0,2.0,4,"{'size': 1820, 'units': 'sqft'}","[{'primary': True, 'advertiser_id': '3384853',...","{'id': '39440415068db4623f0b1b8dda1dbb62', 'na...",2020-05-07T18:40:08Z,"{'presentation_status': 'for_sale', 'is_showca...","{'form': {'name': {'required': True, 'minimum_...",40,https://ap.rdcpix.com/0e43b17238993acc6b4ce5a1...,1,5,type|property|data|prop_id|2263164590|list_id|...,"{'size': 5941, 'units': 'sqft'}","{'name': 'CRMLS', 'id': 'DW20084590', 'plan_id...",NaN


In [5]:
la_listings = CITY_NAME_DF[['property_id', 'prop_type', 'address', 'price', 'baths_half', 'baths_full', 'beds', 'building_size', 'lot_size']]
la_listings["baths_half"] = la_listings["baths_half"].fillna(0)
la_listings

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size
0,M1947154472,single_family,"{'city': 'Los Angeles', 'line': '6207 Keniston...",850000,0.0,2.0,3,"{'size': 1425, 'units': 'sqft'}","{'size': 5649, 'units': 'sqft'}"
1,M1877462085,single_family,"{'city': 'Los Angeles', 'line': '5607 S Wilton...",624900,0.0,2.0,3,"{'size': 1268, 'units': 'sqft'}","{'size': 5470, 'units': 'sqft'}"
2,M2313200155,single_family,"{'city': 'Sun Valley', 'line': '9169 Oneida Av...",610000,0.0,2.0,3,"{'size': 1300, 'units': 'sqft'}","{'size': 10487, 'units': 'sqft'}"
3,M2358860381,single_family,"{'city': 'West Los Angeles', 'line': '11230 Pi...",1999000,0.0,4.0,3,"{'size': 2573, 'units': 'sqft'}","{'size': 6026, 'units': 'sqft'}"
4,M2263164590,single_family,"{'city': 'Los Angeles', 'line': '5011 S Budlon...",739000,0.0,2.0,4,"{'size': 1820, 'units': 'sqft'}","{'size': 5941, 'units': 'sqft'}"
...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,"{'city': 'Los Angeles', 'line': '2131 Baxter S...",1459000,1.0,2.0,4,"{'size': 2368, 'units': 'sqft'}","{'size': 8259, 'units': 'sqft'}"
196,M2230201900,single_family,"{'city': 'Los Angeles', 'line': '554 N Irving ...",1225000,0.0,1.0,2,"{'size': 1329, 'units': 'sqft'}","{'size': 5100, 'units': 'sqft'}"
197,M2062419375,single_family,"{'city': 'Los Angeles', 'line': '8270 Woodshil...",5499000,1.0,5.0,4,"{'size': 4280, 'units': 'sqft'}","{'size': 4835, 'units': 'sqft'}"
198,M1003734243,single_family,"{'city': 'Los Angeles', 'line': '1503 W 66th S...",599000,0.0,2.0,3,"{'size': 1440, 'units': 'sqft'}","{'size': 3927, 'units': 'sqft'}"


In [6]:
from pandas.io.json import json_normalize
la_address = json_normalize(la_listings['address'])
la_address

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,is_approximate,time_zone
0,Los Angeles,6207 Keniston Ave,90043,CA,California,Los Angeles,06037,33.983361,-118.342055,Hyde Park,NaN,NaN
1,Los Angeles,5607 S Wilton Pl,90062,CA,California,Los Angeles,06037,33.991241,-118.315248,South LA,NaN,NaN
2,Sun Valley,9169 Oneida Ave,91352,CA,California,Los Angeles,06037,34.235919,-118.401013,North Valley,NaN,NaN
3,West Los Angeles,11230 Pickford St,90064,CA,California,Los Angeles,06037,34.032995,-118.433781,West Los Angeles,NaN,NaN
4,Los Angeles,5011 S Budlong Ave,90037,CA,California,Los Angeles,06037,33.997393,-118.296193,Vermont Square,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Los Angeles,2131 Baxter St,90039,CA,California,Los Angeles,06037,34.092231,-118.255851,Central LA,NaN,NaN
196,Los Angeles,554 N Irving Blvd,90004,CA,California,Los Angeles,06037,34.080577,-118.318478,Greater Wilshire,NaN,NaN
197,Los Angeles,8270 Woodshill Trl,90069,CA,California,Los Angeles,06037,34.098705,-118.369137,West Los Angeles,NaN,NaN
198,Los Angeles,1503 W 66th St,90047,CA,California,Los Angeles,06037,33.979341,-118.302576,South LA,NaN,NaN


In [7]:
la_df = pd.concat([la_listings,la_address],axis=1)
la_df.dropna(subset=['lot_size','building_size'], inplace=True)
la_df

,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,is_approximate,time_zone
0,M1947154472,single_family,"{'city': 'Los Angeles', 'line': '6207 Keniston...",850000,0.0,2.0,3,"{'size': 1425, 'units': 'sqft'}","{'size': 5649, 'units': 'sqft'}",Los Angeles,6207 Keniston Ave,90043,CA,California,Los Angeles,06037,33.983361,-118.342055,Hyde Park,NaN,NaN
1,M1877462085,single_family,"{'city': 'Los Angeles', 'line': '5607 S Wilton...",624900,0.0,2.0,3,"{'size': 1268, 'units': 'sqft'}","{'size': 5470, 'units': 'sqft'}",Los Angeles,5607 S Wilton Pl,90062,CA,California,Los Angeles,06037,33.991241,-118.315248,South LA,NaN,NaN
2,M2313200155,single_family,"{'city': 'Sun Valley', 'line': '9169 Oneida Av...",610000,0.0,2.0,3,"{'size': 1300, 'units': 'sqft'}","{'size': 10487, 'units': 'sqft'}",Sun Valley,9169 Oneida Ave,91352,CA,California,Los Angeles,06037,34.235919,-118.401013,North Valley,NaN,NaN
3,M2358860381,single_family,"{'city': 'West Los Angeles', 'line': '11230 Pi...",1999000,0.0,4.0,3,"{'size': 2573, 'units': 'sqft'}","{'size': 6026, 'units': 'sqft'}",West Los Angeles,11230 Pickford St,90064,CA,California,Los Angeles,06037,34.032995,-118.433781,West Los Angeles,NaN,NaN
4,M2263164590,single_family,"{'city': 'Los Angeles', 'line': '5011 S Budlon...",739000,0.0,2.0,4,"{'size': 1820, 'units': 'sqft'}","{'size': 5941, 'units': 'sqft'}",Los Angeles,5011 S Budlong Ave,90037,CA,California,Los Angeles,06037,33.997393,-118.296193,Vermont Square,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,"{'city': 'Los Angeles', 'line': '2131 Baxter S...",1459000,1.0,2.0,4,"{'size': 2368, 'units': 'sqft'}","{'size': 8259, 'units': 'sqft'}",Los Angeles,2131 Baxter St,90039,CA,California,Los Angeles,06037,34.092231,-118.255851,Central LA,NaN,NaN
196,M2230201900,single_family,"{'city': 'Los Angeles', 'line': '554 N Irving ...",1225000,0.0,1.0,2,"{'size': 1329, 'units': 'sqft'}","{'size': 5100, 'units': 'sqft'}",Los Angeles,554 N Irving Blvd,90004,CA,California,Los Angeles,06037,34.080577,-118.318478,Greater Wilshire,NaN,NaN
197,M2062419375,single_family,"{'city': 'Los Angeles', 'line': '8270 Woodshil...",5499000,1.0,5.0,4,"{'size': 4280, 'units': 'sqft'}","{'size': 4835, 'units': 'sqft'}",Los Angeles,8270 Woodshill Trl,90069,CA,California,Los Angeles,06037,34.098705,-118.369137,West Los Angeles,NaN,NaN
198,M1003734243,single_family,"{'city': 'Los Angeles', 'line': '1503 W 66th S...",599000,0.0,2.0,3,"{'size': 1440, 'units': 'sqft'}","{'size': 3927, 'units': 'sqft'}",Los Angeles,1503 W 66th St,90047,CA,California,Los Angeles,06037,33.979341,-118.302576,South LA,NaN,NaN


In [8]:
building_size = json_normalize(la_df['building_size'])
building_size

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,size,units
0,1425,sqft
1,1268,sqft
2,1300,sqft
3,2573,sqft
4,1820,sqft
...,...,...
191,2368,sqft
192,1329,sqft
193,4280,sqft
194,1440,sqft


In [9]:
lot_size = json_normalize(la_df['lot_size'])
lot_size

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,size,units
0,5649,sqft
1,5470,sqft
2,10487,sqft
3,6026,sqft
4,5941,sqft
...,...,...
191,8259,sqft
192,5100,sqft
193,4835,sqft
194,3927,sqft


In [10]:
la_single_fam_df = pd.concat([la_listings, la_address, building_size],axis=1)
la_single_fam_df

,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,is_approximate,time_zone,size,units
0,M1947154472,single_family,"{'city': 'Los Angeles', 'line': '6207 Keniston...",850000,0.0,2.0,3,"{'size': 1425, 'units': 'sqft'}","{'size': 5649, 'units': 'sqft'}",Los Angeles,6207 Keniston Ave,90043,CA,California,Los Angeles,06037,33.983361,-118.342055,Hyde Park,NaN,NaN,1425.0,sqft
1,M1877462085,single_family,"{'city': 'Los Angeles', 'line': '5607 S Wilton...",624900,0.0,2.0,3,"{'size': 1268, 'units': 'sqft'}","{'size': 5470, 'units': 'sqft'}",Los Angeles,5607 S Wilton Pl,90062,CA,California,Los Angeles,06037,33.991241,-118.315248,South LA,NaN,NaN,1268.0,sqft
2,M2313200155,single_family,"{'city': 'Sun Valley', 'line': '9169 Oneida Av...",610000,0.0,2.0,3,"{'size': 1300, 'units': 'sqft'}","{'size': 10487, 'units': 'sqft'}",Sun Valley,9169 Oneida Ave,91352,CA,California,Los Angeles,06037,34.235919,-118.401013,North Valley,NaN,NaN,1300.0,sqft
3,M2358860381,single_family,"{'city': 'West Los Angeles', 'line': '11230 Pi...",1999000,0.0,4.0,3,"{'size': 2573, 'units': 'sqft'}","{'size': 6026, 'units': 'sqft'}",West Los Angeles,11230 Pickford St,90064,CA,California,Los Angeles,06037,34.032995,-118.433781,West Los Angeles,NaN,NaN,2573.0,sqft
4,M2263164590,single_family,"{'city': 'Los Angeles', 'line': '5011 S Budlon...",739000,0.0,2.0,4,"{'size': 1820, 'units': 'sqft'}","{'size': 5941, 'units': 'sqft'}",Los Angeles,5011 S Budlong Ave,90037,CA,California,Los Angeles,06037,33.997393,-118.296193,Vermont Square,NaN,NaN,1820.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,"{'city': 'Los Angeles', 'line': '2131 Baxter S...",1459000,1.0,2.0,4,"{'size': 2368, 'units': 'sqft'}","{'size': 8259, 'units': 'sqft'}",Los Angeles,2131 Baxter St,90039,CA,California,Los Angeles,06037,34.092231,-118.255851,Central LA,NaN,NaN,2600.0,sqft
196,M2230201900,single_family,"{'city': 'Los Angeles', 'line': '554 N Irving ...",1225000,0.0,1.0,2,"{'size': 1329, 'units': 'sqft'}","{'size': 5100, 'units': 'sqft'}",Los Angeles,554 N Irving Blvd,90004,CA,California,Los Angeles,06037,34.080577,-118.318478,Greater Wilshire,NaN,NaN,NaN,NaN
197,M2062419375,single_family,"{'city': 'Los Angeles', 'line': '8270 Woodshil...",5499000,1.0,5.0,4,"{'size': 4280, 'units': 'sqft'}","{'size': 4835, 'units': 'sqft'}",Los Angeles,8270 Woodshill Trl,90069,CA,California,Los Angeles,06037,34.098705,-118.369137,West Los Angeles,NaN,NaN,NaN,NaN
198,M1003734243,single_family,"{'city': 'Los Angeles', 'line': '1503 W 66th S...",599000,0.0,2.0,3,"{'size': 1440, 'units': 'sqft'}","{'size': 3927, 'units': 'sqft'}",Los Angeles,1503 W 66th St,90047,CA,California,Los Angeles,06037,33.979341,-118.302576,South LA,NaN,NaN,NaN,NaN


In [11]:
la_single_fam_df = pd.DataFrame(la_single_fam_df)
la_single_fam_df

,property_id,prop_type,address,price,baths_half,baths_full,beds,building_size,lot_size,city,line,postal_code,state_code,state,county,fips_code,lat,lon,neighborhood_name,is_approximate,time_zone,size,units
0,M1947154472,single_family,"{'city': 'Los Angeles', 'line': '6207 Keniston...",850000,0.0,2.0,3,"{'size': 1425, 'units': 'sqft'}","{'size': 5649, 'units': 'sqft'}",Los Angeles,6207 Keniston Ave,90043,CA,California,Los Angeles,06037,33.983361,-118.342055,Hyde Park,NaN,NaN,1425.0,sqft
1,M1877462085,single_family,"{'city': 'Los Angeles', 'line': '5607 S Wilton...",624900,0.0,2.0,3,"{'size': 1268, 'units': 'sqft'}","{'size': 5470, 'units': 'sqft'}",Los Angeles,5607 S Wilton Pl,90062,CA,California,Los Angeles,06037,33.991241,-118.315248,South LA,NaN,NaN,1268.0,sqft
2,M2313200155,single_family,"{'city': 'Sun Valley', 'line': '9169 Oneida Av...",610000,0.0,2.0,3,"{'size': 1300, 'units': 'sqft'}","{'size': 10487, 'units': 'sqft'}",Sun Valley,9169 Oneida Ave,91352,CA,California,Los Angeles,06037,34.235919,-118.401013,North Valley,NaN,NaN,1300.0,sqft
3,M2358860381,single_family,"{'city': 'West Los Angeles', 'line': '11230 Pi...",1999000,0.0,4.0,3,"{'size': 2573, 'units': 'sqft'}","{'size': 6026, 'units': 'sqft'}",West Los Angeles,11230 Pickford St,90064,CA,California,Los Angeles,06037,34.032995,-118.433781,West Los Angeles,NaN,NaN,2573.0,sqft
4,M2263164590,single_family,"{'city': 'Los Angeles', 'line': '5011 S Budlon...",739000,0.0,2.0,4,"{'size': 1820, 'units': 'sqft'}","{'size': 5941, 'units': 'sqft'}",Los Angeles,5011 S Budlong Ave,90037,CA,California,Los Angeles,06037,33.997393,-118.296193,Vermont Square,NaN,NaN,1820.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,"{'city': 'Los Angeles', 'line': '2131 Baxter S...",1459000,1.0,2.0,4,"{'size': 2368, 'units': 'sqft'}","{'size': 8259, 'units': 'sqft'}",Los Angeles,2131 Baxter St,90039,CA,California,Los Angeles,06037,34.092231,-118.255851,Central LA,NaN,NaN,2600.0,sqft
196,M2230201900,single_family,"{'city': 'Los Angeles', 'line': '554 N Irving ...",1225000,0.0,1.0,2,"{'size': 1329, 'units': 'sqft'}","{'size': 5100, 'units': 'sqft'}",Los Angeles,554 N Irving Blvd,90004,CA,California,Los Angeles,06037,34.080577,-118.318478,Greater Wilshire,NaN,NaN,NaN,NaN
197,M2062419375,single_family,"{'city': 'Los Angeles', 'line': '8270 Woodshil...",5499000,1.0,5.0,4,"{'size': 4280, 'units': 'sqft'}","{'size': 4835, 'units': 'sqft'}",Los Angeles,8270 Woodshill Trl,90069,CA,California,Los Angeles,06037,34.098705,-118.369137,West Los Angeles,NaN,NaN,NaN,NaN
198,M1003734243,single_family,"{'city': 'Los Angeles', 'line': '1503 W 66th S...",599000,0.0,2.0,3,"{'size': 1440, 'units': 'sqft'}","{'size': 3927, 'units': 'sqft'}",Los Angeles,1503 W 66th St,90047,CA,California,Los Angeles,06037,33.979341,-118.302576,South LA,NaN,NaN,NaN,NaN


In [0]:
la_single_fam_df.drop(['address', 'building_size', 'state', 'fips_code', 'lat', 'lon', 'neighborhood_name', 'is_approximate', 'time_zone'], axis=1, inplace=True)

In [0]:
del la_single_fam_df['units']

In [0]:
la_single_fam_df.rename(columns={'size': "building_size"}, inplace=True)

In [21]:
la_single_fam_df

,property_id,prop_type,price,baths_half,baths_full,beds,lot_size,city,line,postal_code,state_code,county,building_size
0,M1947154472,single_family,850000,0.0,2.0,3,"{'size': 5649, 'units': 'sqft'}",Los Angeles,6207 Keniston Ave,90043,CA,Los Angeles,1425.0
1,M1877462085,single_family,624900,0.0,2.0,3,"{'size': 5470, 'units': 'sqft'}",Los Angeles,5607 S Wilton Pl,90062,CA,Los Angeles,1268.0
2,M2313200155,single_family,610000,0.0,2.0,3,"{'size': 10487, 'units': 'sqft'}",Sun Valley,9169 Oneida Ave,91352,CA,Los Angeles,1300.0
3,M2358860381,single_family,1999000,0.0,4.0,3,"{'size': 6026, 'units': 'sqft'}",West Los Angeles,11230 Pickford St,90064,CA,Los Angeles,2573.0
4,M2263164590,single_family,739000,0.0,2.0,4,"{'size': 5941, 'units': 'sqft'}",Los Angeles,5011 S Budlong Ave,90037,CA,Los Angeles,1820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,1459000,1.0,2.0,4,"{'size': 8259, 'units': 'sqft'}",Los Angeles,2131 Baxter St,90039,CA,Los Angeles,2600.0
196,M2230201900,single_family,1225000,0.0,1.0,2,"{'size': 5100, 'units': 'sqft'}",Los Angeles,554 N Irving Blvd,90004,CA,Los Angeles,NaN
197,M2062419375,single_family,5499000,1.0,5.0,4,"{'size': 4835, 'units': 'sqft'}",Los Angeles,8270 Woodshill Trl,90069,CA,Los Angeles,NaN
198,M1003734243,single_family,599000,0.0,2.0,3,"{'size': 3927, 'units': 'sqft'}",Los Angeles,1503 W 66th St,90047,CA,Los Angeles,NaN


In [22]:
la_single_fam_df2 = pd.concat([la_single_fam_df, lot_size],axis=1)
la_single_fam_df2

,property_id,prop_type,price,baths_half,baths_full,beds,lot_size,city,line,postal_code,state_code,county,building_size,size,units
0,M1947154472,single_family,850000,0.0,2.0,3,"{'size': 5649, 'units': 'sqft'}",Los Angeles,6207 Keniston Ave,90043,CA,Los Angeles,1425.0,5649.0,sqft
1,M1877462085,single_family,624900,0.0,2.0,3,"{'size': 5470, 'units': 'sqft'}",Los Angeles,5607 S Wilton Pl,90062,CA,Los Angeles,1268.0,5470.0,sqft
2,M2313200155,single_family,610000,0.0,2.0,3,"{'size': 10487, 'units': 'sqft'}",Sun Valley,9169 Oneida Ave,91352,CA,Los Angeles,1300.0,10487.0,sqft
3,M2358860381,single_family,1999000,0.0,4.0,3,"{'size': 6026, 'units': 'sqft'}",West Los Angeles,11230 Pickford St,90064,CA,Los Angeles,2573.0,6026.0,sqft
4,M2263164590,single_family,739000,0.0,2.0,4,"{'size': 5941, 'units': 'sqft'}",Los Angeles,5011 S Budlong Ave,90037,CA,Los Angeles,1820.0,5941.0,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M2414866018,single_family,1459000,1.0,2.0,4,"{'size': 8259, 'units': 'sqft'}",Los Angeles,2131 Baxter St,90039,CA,Los Angeles,2600.0,6077.0,sqft
196,M2230201900,single_family,1225000,0.0,1.0,2,"{'size': 5100, 'units': 'sqft'}",Los Angeles,554 N Irving Blvd,90004,CA,Los Angeles,NaN,NaN,NaN
197,M2062419375,single_family,5499000,1.0,5.0,4,"{'size': 4835, 'units': 'sqft'}",Los Angeles,8270 Woodshill Trl,90069,CA,Los Angeles,NaN,NaN,NaN
198,M1003734243,single_family,599000,0.0,2.0,3,"{'size': 3927, 'units': 'sqft'}",Los Angeles,1503 W 66th St,90047,CA,Los Angeles,NaN,NaN,NaN


In [0]:
la_single_fam_df2 = pd.DataFrame(la_single_fam_df2)

In [0]:
del la_single_fam_df2['lot_size']

In [0]:
del la_single_fam_df2['units']

In [0]:
la_single_fam_df2.rename(columns={'size': "lot_size"}, inplace=True)

In [29]:
la_single_fam_df2.dropna(inplace=True)
la_single_fam_df2

,property_id,prop_type,price,baths_half,baths_full,beds,city,line,postal_code,state_code,county,building_size,lot_size
0,M1947154472,single_family,850000,0.0,2.0,3,Los Angeles,6207 Keniston Ave,90043,CA,Los Angeles,1425.0,5649.0
1,M1877462085,single_family,624900,0.0,2.0,3,Los Angeles,5607 S Wilton Pl,90062,CA,Los Angeles,1268.0,5470.0
2,M2313200155,single_family,610000,0.0,2.0,3,Sun Valley,9169 Oneida Ave,91352,CA,Los Angeles,1300.0,10487.0
3,M2358860381,single_family,1999000,0.0,4.0,3,West Los Angeles,11230 Pickford St,90064,CA,Los Angeles,2573.0,6026.0
4,M2263164590,single_family,739000,0.0,2.0,4,Los Angeles,5011 S Budlong Ave,90037,CA,Los Angeles,1820.0,5941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,M2055450644,single_family,1269888,1.0,2.0,3,Los Angeles,2266 Moss Ave,90065,CA,Los Angeles,2368.0,8259.0
192,M1431994454,single_family,520000,0.0,1.0,3,Los Angeles,922 E 85th St,90001,CA,Los Angeles,1329.0,5100.0
193,M2287573782,single_family,1495000,0.0,28.0,15,Los Angeles,532 W Vernon Ave,90037,CA,Los Angeles,4280.0,4835.0
194,M2890808965,single_family,2995000,0.0,3.0,4,Pacific Palisades,16733 Bollinger Dr,90272,CA,Los Angeles,1440.0,3927.0


In [30]:
la_single_fam_final = pd.DataFrame(la_single_fam_df2)
la_single_fam_final

,property_id,prop_type,price,baths_half,baths_full,beds,city,line,postal_code,state_code,county,building_size,lot_size
0,M1947154472,single_family,850000,0.0,2.0,3,Los Angeles,6207 Keniston Ave,90043,CA,Los Angeles,1425.0,5649.0
1,M1877462085,single_family,624900,0.0,2.0,3,Los Angeles,5607 S Wilton Pl,90062,CA,Los Angeles,1268.0,5470.0
2,M2313200155,single_family,610000,0.0,2.0,3,Sun Valley,9169 Oneida Ave,91352,CA,Los Angeles,1300.0,10487.0
3,M2358860381,single_family,1999000,0.0,4.0,3,West Los Angeles,11230 Pickford St,90064,CA,Los Angeles,2573.0,6026.0
4,M2263164590,single_family,739000,0.0,2.0,4,Los Angeles,5011 S Budlong Ave,90037,CA,Los Angeles,1820.0,5941.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,M2055450644,single_family,1269888,1.0,2.0,3,Los Angeles,2266 Moss Ave,90065,CA,Los Angeles,2368.0,8259.0
192,M1431994454,single_family,520000,0.0,1.0,3,Los Angeles,922 E 85th St,90001,CA,Los Angeles,1329.0,5100.0
193,M2287573782,single_family,1495000,0.0,28.0,15,Los Angeles,532 W Vernon Ave,90037,CA,Los Angeles,4280.0,4835.0
194,M2890808965,single_family,2995000,0.0,3.0,4,Pacific Palisades,16733 Bollinger Dr,90272,CA,Los Angeles,1440.0,3927.0


In [32]:
from google.colab import drive
drive.mount('/drive')
la_single_fam_final.to_csv('/drive/My Drive/UCI-IRV-2020-04/la_single_family_listings.csv')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive
